In [115]:
from openai import OpenAI

In [116]:
from dotenv import load_dotenv

In [117]:
load_dotenv()

True

In [118]:
import os

In [119]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [120]:
from datetime import datetime

In [121]:
import requests

In [122]:
from supabase import create_client, Client

In [94]:
supabase_url = os.environ.get("SUPABASE_URL")

In [95]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [96]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [103]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls, news_date')\
.eq('news_date', '2026-02-20')\
    .order('created_at', desc=False)\
    .execute()

result.data


[{'id': 1273,
  'news_output': 'Unicity Labs announced a $3M funding raise to scale autonomous agentic marketplaces — a funding event in the AI/agent space (press release).',
  'source_urls': ['https://en.prnasia.com/releases/apac/unicity-labs-raises-3m-to-scale-autonomous-agentic-marketplaces-522807.shtml'],
  'news_date': '2026-02-20'},
 {'id': 1274,
  'news_output': 'Reuters: Nvidia is reported to be close to finalizing roughly a $30 billion investment into OpenAI (reported as a near-term investment arrangement replacing an unfinished larger deal), per Reuters coverage of Feb 20, 2026. This is a major funding/deal development in AI finance and investor landscape.',
  'source_urls': ['https://www.reuters.com/business/nvidia-close-finalizing-30-billion-investment-openai-funding-round-ft-reports-2026-02-20/'],
  'news_date': '2026-02-20'},
 {'id': 1275,
  'news_output': 'Industry/press item: OpenAI and Tata Group announced a strategic AI partnership in India covering skills, enterprise

In [112]:
a = [{'id': 1274,
  'news_output': 'Reuters: Nvidia is reported to be close to finalizing roughly a $30 billion investment into OpenAI (reported as a near-term investment arrangement replacing an unfinished larger deal), per Reuters coverage of Feb 20, 2026. This is a major funding/deal development in AI finance and investor landscape.',
  'source_urls': ['https://www.reuters.com/business/nvidia-close-finalizing-30-billion-investment-openai-funding-round-ft-reports-2026-02-20/'],
  'news_date': '2026-02-20'},
 {'id': 1275,
  'news_output': 'Industry/press item: OpenAI and Tata Group announced a strategic AI partnership in India covering skills, enterprise ChatGPT, agentic AI and infrastructure (reported as a new multi-layered deal/partnership announcement on Feb 20, 2026). This is a material commercial/partnership development for distribution and infrastructure in a major market.',
  'source_urls': ['https://www.edtechinnovationhub.com/news/openai-and-tata-group-forge-multi-layered-ai-partnership-spanning-skills-enterprise-and-infrastructure'],
  'news_date': '2026-02-20'},
 {'id': 1276,
  'news_output': 'General Catalyst announced a $5 billion commitment to invest in India over the next five years — a major VC commitment announced at the India AI Impact Summit 2026 (intended to back startups and scale-ups in India).',
  'source_urls': ['https://www.cnbctv18.com/business/anthropic-discord-and-stripe-investor-general-catalyst-commits-5-billion-to-india-ws-l-19854554.htm'],
  'news_date': '2026-02-20'},
 {'id': 1277,
  'news_output': 'Abu Dhabi’s G42 and MBZUAI, together with US firm Cerebras and India’s Centre for Development of Advanced Computing, announced a plan to establish a national-scale AI supercomputer in India (reports cite an 8 exaflop-class deployment) — an infrastructure partnership announced at the India AI Impact Summit.',
  'source_urls': ['https://www.mediaoffice.abudhabi/en/technology/g42-mohamed-bin-zayed-university-of-artificial-intelligence-and-cerebras-partner-with-centre-for-development-of-advanced-computing-to-establish-ai-supercomputer-in-india/'],
  'news_date': '2026-02-20'},
 {'id': 1278,
  'news_output': 'Industry coverage (BW Disrupt): Sarvam AI announced two new large language models designed for real‑time use and advanced reasoning — a new-model announcement from a specialized LLM provider.',
  'source_urls': ['https://www.bwdisrupt.com/article/sarvam-ai-unveils-two-new-large-language-models-designed-for-real-time-use-advanced-reasoning-594437'],
  'news_date': '2026-02-20'},
 {'id': 1279,
  'news_output': 'Fractal (press release reported on PR Newswire): Fractal launched Vaidya 2.0, claiming improved performance on healthcare AI benchmarks and positioning it against leading frontier models for health applications — a product/model update focused on a vertical use case.',
  'source_urls': ['https://www.prnewswire.com/in/news-releases/fractal-launches-vaidya-2-0--outperforming-leading-frontier-models-on-healthcare-ai-benchmarks-302692635.html'],
  'news_date': '2026-02-20'},
 {'id': 1280,
  'news_output': 'Stanford HAI and AWS launched an AI marketing science lab focused on measurement, causal inference, and applied AI measurement research — an academic‑industry initiative relevant to AI product measurement and adoption.',
  'source_urls': ['https://www.edtechinnovationhub.com/news/stanford-hai-and-aws-launch-marketing-science-lab-focused-on-ai-measurement'],
  'news_date': '2026-02-20'},
 {'id': 1284,
  'news_output': 'Google Cloud (official blog) announced Gemini 3.1 Pro — the Gemini family update made available on Gemini CLI, Gemini Enterprise, and Vertex AI, billed as a more capable enterprise-grade model and tooling update.',
  'source_urls': ['https://cloud.google.com/blog/products/ai-machine-learning/gemini-3-1-pro-on-gemini-cli-gemini-enterprise-and-vertex-ai'],
  'news_date': '2026-02-20'},
 {'id': 1286,
  'news_output': 'OpenAI and Microsoft joined a UK-led international coalition to safeguard AI development, a coordination initiative reported by news agencies; this is a policy/governance development involving two major AI companies.',
  'source_urls': ['https://www.aninews.in/news/business/openai-microsoft-join-uk-led-global-coalition-to-safeguard-ai-development20260220141850'],
  'news_date': '2026-02-20'},
 {'id': 1287,
  'news_output': 'PhonePe rolled out a new AI-powered feature allowing users to pay via voice or text commands inside the app — a product feature launch from a major Indian fintech.',
  'source_urls': ['https://m.economictimes.com/industry/banking/finance/phonepe-rolls-out-ai-powered-feature-to-pay-via-voice-or-text/articleshow/128601184.cms'],
  'news_date': '2026-02-20'},
 {'id': 1288,
  'news_output': "Forbes (analysis) reported on OpenAI releasing a new 'ChatGPT Lockdown Mode' intended to limit problematic outputs (e.g., risky mental-health advice) — a notable product/behavioral-safety change for ChatGPT reported on 2026-02-20.",
  'source_urls': ['https://www.forbes.com/sites/lanceeliot/2026/02/20/new-chatgpt-lockdown-mode-repurposed-to-stop-ai-from-giving-out-bad-mental-health-advice/'],
  'news_date': '2026-02-20'},
 {'id': 1289,
  'news_output': 'Republic World reported Meta plans to launch an AI-powered smartwatch in late 2026 with health tracking and a built-in assistant — a product roadmap/announcement relevant to AI-enabled device trends.',
  'source_urls': ['http://www.republicworld.com/tech/meta-to-launch-ai-powered-smartwatch-in-late-2026-with-health-tracking-and-built-in-assistant'],
  'news_date': '2026-02-20'},
 {'id': 1291,
  'news_output': "Google DeepMind announced (and outlets report) Lyria 3, a new generative music model, is rolling out in beta inside the Gemini app — enabling users to generate short music tracks from text, photo or video prompts, a notable product feature expansion for Gemini's multimodal capabilities.",
  'source_urls': ['https://www.ndtv.com/world-news/lyria-3-all-about-googles-ai-powered-tool-that-lets-you-create-song-from-text-and-videos-11113239'],
  'news_date': '2026-02-20'},
 {'id': 1292,
  'news_output': 'Commercial insurtech Qumis raised $4.3 million to scale its AI platform for commercial insurance underwriting and distribution — a modest but relevant funding event within the sector that signals continued VC interest in verticalized AI insurance tooling.',
  'source_urls': ['https://fintech.global/2026/02/20/commercial-insurtech-qumis-raises-4-3m-to-scale-ai-platform/'],
  'news_date': '2026-02-20'},
 {'id': 1294,
  'news_output': 'Reuters reports OpenAI is developing a family of AI-powered devices — including a smart speaker and possibly smart glasses — with 200+ people working on the effort; signals OpenAI moving from solely cloud/model products toward consumer device integration.',
  'source_urls': ['https://www.reuters.com/business/openai-developing-ai-devices-including-smart-speaker-information-reports-2026-02-20/'],
  'news_date': '2026-02-20'}]


In [111]:
len(a)

16

In [108]:
b = []

for i in a:
    current_news = i['news_output']

    if not any(item['news_output'] == current_news for item in b):
        b.append(i)
b

[{'id': 1273,
  'news_output': 'Unicity Labs announced a $3M funding raise to scale autonomous agentic marketplaces — a funding event in the AI/agent space (press release).',
  'source_urls': ['https://en.prnasia.com/releases/apac/unicity-labs-raises-3m-to-scale-autonomous-agentic-marketplaces-522807.shtml'],
  'news_date': '2026-02-20'},
 {'id': 1274,
  'news_output': 'Reuters: Nvidia is reported to be close to finalizing roughly a $30 billion investment into OpenAI (reported as a near-term investment arrangement replacing an unfinished larger deal), per Reuters coverage of Feb 20, 2026. This is a major funding/deal development in AI finance and investor landscape.',
  'source_urls': ['https://www.reuters.com/business/nvidia-close-finalizing-30-billion-investment-openai-funding-round-ft-reports-2026-02-20/'],
  'news_date': '2026-02-20'},
 {'id': 1275,
  'news_output': 'Industry/press item: OpenAI and Tata Group announced a strategic AI partnership in India covering skills, enterprise

In [109]:
c = [{'id': 671,
  'news_output': "Meta and NVIDIA announced a new multiyear, multigenerational strategic infrastructure partnership to deploy NVIDIA Blackwell/Rubin GPUs and support Meta's AI-optimized data centers — a large-scale infrastructure pact announced Feb 18, 2026.",
  'source_urls': ['https://about.fb.com/news/2026/02/meta-nvidia-announce-long-term-infrastructure-partnership/'],
  'news_date': '2026-02-18'},
 {'id': 678,
  'news_output': 'Gartner published a Feb 18, 2026 press release reporting a survey where 91% of customer service leaders feel pressure to implement AI in 2026 and that a majority plan to reshape frontline roles around AI.',
  'source_urls': ['https://www.gartner.com/en/newsroom/press-releases/2026-02-18-gartner-survey-finds-ninety-one-percent-of-customer-service-leaders-under-pressure-to-implement-ai-in-2026'],
  'news_date': '2026-02-18'},
 {'id': 680,
  'news_output': 'Anthropic announced Claude Sonnet 4.6 (published Feb 18, 2026): Sonnet 4.6 brings improvements over Sonnet 4.5 in coding, long-context reasoning, agent planning, knowledge work and design, and Anthropic said it is testing a 1,000,000-token context window in beta to support much larger single-session contexts.',
  'source_urls': ['https://www.business-standard.com/technology/tech-news/anthropic-launched-claude-sonnet-4-6-model-features-improvements-availability-126021800630_1.html'],
  'news_date': '2026-02-18'},
 {'id': 681,
  'news_output': "Kana (marketing AI startup) raised $15 million in seed funding (published Feb 18, 2026): round led by Mayfield to develop configurable 'loosely coupled' AI agents for marketers that integrate with existing stacks, with an emphasis on human-in-the-loop workflows and synthetic-data capabilities.",
  'source_urls': ['https://ventureburn.com/kana-raises-15m-to-power-flexible-ai-for-marketers/'],
  'news_date': '2026-02-18'},
 {'id': 682,
  'news_output': "Economic Times / Financial Times reporting: David Silver's London-based startup Ineffable Intelligence is reportedly close to raising a ~$1 billion seed round led by Sequoia (reported Feb 18, 2026); sources say potential interest from Nvidia, Google and Microsoft — a significant rumored early-stage mega-round.",
  'source_urls': ['https://m.economictimes.com/tech/artificial-intelligence/former-google-deepmind-scientist-to-raise-1-billion-led-by-sequoia-for-superhuman-intelligence/articleshow/128513567.cms'],
  'news_date': '2026-02-18'},
 {'id': 683,
  'news_output': 'Announcements at India AI Impact Summit (Feb 18, 2026): NPCI announced a collaboration with Nvidia to build a payments-native AI layer for India, and Yotta/Nvidia outlined deploying a large DGX Cloud cluster in India (APAC DGX Cloud expansion) — regional AI compute and payments infrastructure moves that could materially affect AI deployment in India.',
  'source_urls': ['https://www.business-standard.com/technology/tech-news/tech-wrap-feb-18-google-i-o-2026-date-announced-apple-ai-devices-antheopic-claude-sonnet-4-6-google-pixel-10a-ios-26-4-public-beta-126021800987_1.html'],
  'news_date': '2026-02-18'},
 {'id': 685,
  'news_output': "TechCrunch reports World Labs raised $200 million from Autodesk to integrate 'world models' into 3D workflows — a sizable funding round and product-integration partnership.",
  'source_urls': ['https://techcrunch.com/2026/02/18/world-labs-lands-200m-from-autodesk-to-bring-world-models-into-3d-workflows/'],
  'news_date': '2026-02-18'},
 {'id': 686,
  'news_output': 'Zyphra announced ZUNA, a BCI (brain–computer interface) foundation model aimed at advancing thought-to-text capabilities — a new-model release with potential research and product implications.',
  'source_urls': ['https://www.morningstar.com/news/pr-newswire/20260218la90293/zyphra-releases-zuna-bci-foundation-model-advancing-towards-thought-to-text'],
  'news_date': '2026-02-18'},
 {'id': 691,
  'news_output': "Microsoft said it is on pace to invest about $50 billion by the end of the decade to expand AI across countries in the 'Global South' — announcement reported by Reuters on Feb 18, 2026.",
  'source_urls': ['https://www.reuters.com/world/china/microsoft-says-it-is-pace-invest-50-billion-global-south-ai-push-2026-02-18/'],
  'news_date': '2026-02-18'},
 {'id': 695,
  'news_output': 'JFB and XTEND announced a $1.5 billion business combination to form an AI-driven autonomous defense robotics company — a major strategic transaction in the defense/robotics + AI space (announcement dated Feb 18, 2026).',
  'source_urls': ['https://theaiinsider.tech/2026/02/18/jfb-and-xtend-announce-1-5b-business-combination-to-establish-ai-driven-autonomous-defense-robotics-company/'],
  'news_date': '2026-02-18'},
 {'id': 696,
  'news_output': 'CNBC: Anthropic is publicly clashing with the U.S. Department of Defense over permissible uses of its models — Anthropic seeks assurances its models won’t be used for autonomous weapons or mass domestic surveillance; the DOD wants rights to use models for “all lawful use cases.” This dispute could affect Anthropic’s government deployments and sets a notable precedent for how frontier-model vendors negotiate use restrictions with national security customers.',
  'source_urls': ['https://www.cnbc.com/2026/02/18/anthropic-pentagon-ai-defense-war-surveillance.html'],
  'news_date': '2026-02-18'},
 {'id': 700,
  'news_output': "India's data-center operator Yotta announced plans to build a $2 billion AI hub using Nvidia's new Blackwell chips — a major infrastructure investment to host large-scale AI workloads and accelerate regional AI adoption (reported by Reuters).",
  'source_urls': ['https://www.reuters.com/technology/indias-yotta-build-2-billion-ai-hub-with-nvidias-blackwell-chips-2026-02-18/'],
  'news_date': '2026-02-18'},
 {'id': 703,
  'news_output': 'Fortune (exclusive): Cogent Security raised a $42 million Series A; investors include Bain and Greylock. The startup is positioning AI agents to tackle persistent cybersecurity bottlenecks, per the report.',
  'source_urls': ['https://fortune.com/2026/02/18/exclusive-bain-and-greylock-bet-42-million-that-ai-agents-can-finally-fix-cybersecuritys-messiest-bottleneck/'],
  'news_date': '2026-02-18'},
 {'id': 704,
  'news_output': 'Austin Business Journal / BizJournals: Circuit, an AI startup by Tyson Tuttle, raised ~$30 million in a new round from angels and early investors (reported as a strong angel round supporting the company’s initial growth).',
  'source_urls': ['https://www.bizjournals.com/austin/news/2026/02/18/austin-startup-circuit-raises-millions.html'],
  'news_date': '2026-02-18'},
 {'id': 709,
  'news_output': 'OpenAI community post (Feb 18, 2026) announcing GPT-5.3-Codex-Spark research preview: an ultra-fast real-time coding model (reports of ~1000 tokens/sec) rolling out today to ChatGPT Pro users in the Codex app, CLI, and IDE extension — a notable model/feature rollout.',
  'source_urls': ['https://community.openai.com/t/gpt-5-3-codex-spark-research-preview-with-1000-tokens-per-second/1374091'],
  'news_date': '2026-02-18'}]

In [82]:
research_output_7 = []

In [83]:
def openai_research_v2(research_input):
    for i in research_input:
        response = client.responses.create(
            model = "gpt-5-nano",
            tools = [{
                "type": "web_search"
            }],
            include = ["web_search_call.action.sources"],
            input = f"""You are an AI research analyst for Krux.
            Your job is to create a fact-checked research brief for a given AI news event.
            
            APPROACH:
            1. Start with WHAT IS NEW — the specific event, announcement, or development.
            2. Add ONLY the context needed to understand why this matters.
            3. For funding/M&A: focus on the new round, amount, investors, and valuation. 
               Do NOT include historical funding rounds or prior valuations.
            4. For product/launch news: include key technical details, who built it, 
               and what problem it solves.
            5. Maximum 8-10 bullet points. If you need more, you're being redundant.
            
            CRITICAL RULES:
            - Every fact must be traceable to a credible published source.
            - No point should repeat information from another point.
            - No opinions, predictions, or analysis.
            - No fluff — get straight to the facts.
            - First bullet must always state the core new development.
            
            OUTPUT: Bullet points with inline source citations. Nothing else.

            Here is what you need to research on:

            {i['news_output']}
            """,
        )

        output = response.output_text
        print(output)

        final_dictionary = {
            'event_id': f"{i['id']}_{i['news_date']}",
            'news_date': i['news_date'],
            'output': output,
            'model_provider': 'openai'
        }

        research_output_7.append(final_dictionary)
        print(final_dictionary)

In [113]:
openai_research_v2(a)

- Core development: Nvidia is close to finalizing roughly a $30 billion equity investment in OpenAI, replacing the previously announced up-to-$100 billion multi-year data-center compute pact. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))

- Financing scope: OpenAI is seeking more than $100 billion in the current funding round, with the round valuing OpenAI at about $830 billion. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))

- Likely investors: SoftBank and Amazon are also expected to participate in the round. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))

- Investment structure: The deal would be a direct equity stake in OpenAI, not a

In [114]:
for i in research_output_7:
    if i['news_date'] == '2026-02-20':
        save_research(i)
        print("✅ Saved")

✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved


In [24]:
a = [{'event_id': '567_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Core development: The India AI Impact Summit 2026 is scheduled for February 16–20, 2026 in New Delhi, with the Bharat Mandapam Expo Arena and Sushma Swaraj Bhawan as primary venues; Prime Minister Modi is slated to inaugurate the AI Expo on February 16 at 5:00 PM, with the Expo open to the public from February 17. ([impactexpo.indiaai.gov.in](https://www.impactexpo.indiaai.gov.in/?utm_source=openai))\n\n- High-profile attendees: Reported participants include OpenAI CEO Sam Altman and Google CEO Sundar Pichai, among other tech leaders (e.g., Dario Amodei, Demis Hassabis); organizers anticipate 20 heads of state and 40+ CEOs. ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/india-ai-impact-summit-2026-what-it-is-whos-coming-and-why-it-matters/articleshow/128207697.cms?utm_source=openai))\n\n- Attendee-availability uncertainty for Altman: Media reports vary on Altman’s confirmed attendance; some outlets say he is expected to visit, while others cite no confirmed listing. ([economictimes.indiatimes.com](https://economictimes.indiatimes.com/tech/artificial-intelligence/openais-sam-altman-eyes-india-visit-as-global-ai-leaders-gather-in-new-delhi/articleshow/127470231.cms?utm_source=openai))\n\n- Nvidia attendance status: Reports have differed on Nvidia’s leader’s presence; Barron’s notes Jensen Huang is not attending, while other coverage has implied he could be part of the delegation. ([barrons.com](https://www.barrons.com/articles/alphabet-anthropic-stock-ceo-ai-summit-c1833be3?utm_source=openai))\n\n- Summit focus and structure: The event is framed around tangible collaboration and outcomes, organized conceptually around three Sutras (People, Planet, Progress) and seven Chakras, with a goal of moving from pilot projects to policy/action and signaling near-term partnerships and expansions. ([indianexpress.com](https://indianexpress.com/article/explained/explained-sci-tech/ai-impact-summit-begins-delhi-agenda-10533773/?utm_source=openai))\n\n- Global South leadership context: Coverage emphasizes that this is the first major global AI summit hosted in the Global South, signaling India’s role in shaping inclusive AI governance and deployment. ([apnews.com](https://apnews.com/article/f32aa15263349dadad1840dadc75ca83?utm_source=openai))\n\n- Pre-summit activities and governance: The organizers have launched a formal Call for Pre-Summit Events, and there have been multiple stakeholder consultations in 2025 as part of preparing the summit agenda. ([impact.indiaai.gov.in](https://impact.indiaai.gov.in/home/pre-summit-events?utm_source=openai))\n\n- Expo scale and sector focus: The India AI Impact Expo at Bharat Mandapam will feature AI products and solutions across industries (healthcare, education, governance, etc.), with the event highlighting partnerships, deployments, and industry uptake. ([impactexpo.indiaai.gov.in](https://www.impactexpo.indiaai.gov.in/?utm_source=openai))\n\n- Prime Minister Modi’s onsite role and program: In addition to inaugurating the expo, Modi is slated to address the main plenary (Feb 19) and engage with a wide slate of CEOs during the summit. ([m.economictimes.com](https://m.economictimes.com/ai/ai-insights/prime-minister-narendra-modi-inaugurates-india-ai-impact-expo-2026-at-bharat-mandapam-new-delhi-artificial-intelligence/articleshow/128420514.cms?utm_source=openai))',
  'model_provider': 'openai'},
 {'event_id': '585_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Doubao 2.0 launched on February 14, 2026 as an upgrade to ByteDance’s flagship Doubao AI chatbot; the Pro version adds advanced reasoning and multi-step task execution, with ByteDance claiming usage costs per token are reduced by roughly 10x. ([straitstimes.com](https://www.straitstimes.com/asia/east-asia/chinas-bytedance-releases-doubao-2-0-ai-model-for-agent-era//?utm_source=openai))\n\n- The release was timed ahead of the Lunar New Year holiday, a peak period for user engagement in China. ([straitstimes.com](https://www.straitstimes.com/asia/east-asia/chinas-bytedance-releases-doubao-2-0-ai-model-for-agent-era//?utm_source=openai))\n\n- The Pro variant is described as capable of complex reasoning and multi-step task execution that ByteDance positions against leading frontier models (GPT-5.2 and Gemini 3 Pro). ([straitstimes.com](https://www.straitstimes.com/asia/east-asia/chinas-bytedance-releases-doubao-2-0-ai-model-for-agent-era//?utm_source=openai))\n\n- Doubao 2.0 was developed by ByteDance’s Seed research team and built using Volcano Engine API services, with a focus on long-chain reasoning. ([scmp.com](https://www.scmp.com/tech/big-tech/article/3332365/bytedance-unveils-chinas-most-affordable-ai-coding-agent-just-us130-month?utm_source=openai))\n\n- Doubao remains China’s leading AI chatbot by user reach, with approximately 155 million weekly active users as of late December 2025. ([straitstimes.com](https://www.straitstimes.com/asia/east-asia/chinas-bytedance-releases-doubao-2-0-ai-model-for-agent-era//?utm_source=openai))\n\n- ByteDance previously expanded Doubao with Doubao-Seed-Code, a coding agent released in November 2025, priced at 9.9 yuan for the first month and 40 yuan monthly thereafter; it achieved SWE-Bench Verified status. ([scmp.com](https://www.scmp.com/tech/big-tech/article/3332365/bytedance-unveils-chinas-most-affordable-ai-coding-agent-just-us130-month?utm_source=openai))\n\n- Concurrently, ByteDance released Seedance 2.0, a video-generation model, around February 12–13, 2026, which circulated widely online and attracted copyright complaints. ([investing.com](https://www.investing.com/news/stock-market-news/disney-sends-ceaseanddesist-to-bytedance-over-aigenerated-videos-4507348?utm_source=openai))\n\n- The Seedance 2.0 controversy drew cease-and-desist actions from Disney and others, with ByteDance pledging to strengthen safeguards to curb IP risk and unauthorized likeness usage. ([investing.com](https://www.investing.com/news/stock-market-news/disney-sends-ceaseanddesist-to-bytedance-over-aigenerated-videos-4507348?utm_source=openai))\n\n- The IP backlash to Seedance 2.0 occurred within a broader domestic AI-ecosystem competition in China, which included Alibaba’s February 6, 2026 3 billion yuan Qwen coupon campaign to boost in-chat engagement. ([straitstimes.com](https://www.straitstimes.com/asia/east-asia/chinas-bytedance-releases-doubao-2-0-ai-model-for-agent-era//?utm_source=openai))',
  'model_provider': 'openai'},
 {'event_id': '592_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Core new development: Zhipu AI launched its GLM‑5 in overseas markets and announced GLM Coding Plan price adjustments, with domestic package prices rising about 30% (effective February 12, 2026) and overseas Coding Plan/API pricing rising roughly 30%–60% and 67%–100%, respectively. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))\n- Domestic pricing detail: Domestic GLM package prices have increased by at least 30%, and existing subscribers will continue at their current rates; the changes took effect February 12, 2026. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))\n- Overseas pricing detail: Overseas Coding Plan prices are up about 30%–60%, and overseas API usage fees are up about 67%–100%; overseas increases are higher than domestic ones. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))\n- Claimed drivers: TrendForce attributes the price moves to rising demand for GLM‑5 and higher compute costs, contributing to a broader price-stabilization trend among Chinese LLM providers. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))\n- Market context: TrendForce notes a broader price-stabilization trend across Chinese LLM providers, with several firms beginning to halt price cuts in 2025 and pursuing tiered pricing strategies. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))\n- 2025 context referenced: The report mentions that other “Six Rising Tigers” (e.g., Moonshot AI, MiniMax, StepFun) had already raised certain API prices in 2025, while Baichuan Intelligence and 01.AI kept prices unchanged; Alibaba/Tencent/Baidu have been adopting tiered pricing. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))\n- GLM‑5 profile: GLM‑5 is described as Zhipu AI’s next‑gen flagship large language model. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))\n- Coding Plan focus: The GLM Coding Plan targets developer code generation and programming assistance use cases. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))\n- Publication context: TrendForce published these observations on February 16, 2026, summarizing events surrounding GLM‑5’s overseas rollout and price adjustments. ([trendforce.com](https://www.trendforce.com/news/2026/02/16/news-rising-costs-and-demand-drive-chinas-llm-price-jump-zhipu-glm%E2%80%915-hikes-30-in-first-2026-increase/?utm_source=openai))',
  'model_provider': 'openai'},
 {'event_id': '594_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Core development: Blackstone-led financing into Neysa totaling up to $1.2 billion to build India’s AI infrastructure platform, with up to $600 million in equity from Blackstone and co-investors and up to $600 million of debt financing; Blackstone majority stake to be held post-transaction; announcement date February 16, 2026. ([blackstone.com](https://www.blackstone.com/news/press/blackstone-leads-funding-of-over-1-billion-to-neysa-to-work-towards-building-indias-leading-ai-infrastructure-platform/))\n\n- Investors: Equity participants include Blackstone plus co-investors Teachers’ Venture Growth, TVS Capital, 360 ONE Asset, and Nexus Venture Partners; debt component described as up to $600 million, to be raised concurrently. ([techcrunch.com](https://techcrunch.com/2026/02/15/blackstone-backs-neysa-in-up-to-1-2b-financing-as-india-pushes-to-build-domestic-ai-compute/))\n\n- Deal structure: The $1.2 billion capital raise is described as a split between primary equity and debt financing (roughly equal share each), enabling Neysa to deploy larger GPU capacity while Blackstone takes a majority stake. ([techcrunch.com](https://techcrunch.com/2026/02/15/blackstone-backs-neysa-in-up-to-1-2b-financing-as-india-pushes-to-build-domestic-ai-compute/))\n\n- Use of funds: Funds allocated to scale Neysa’s GPU capacity to over 20,000 GPUs in India and accelerate deployment across its AI infrastructure platform. ([blackstone.com](https://www.blackstone.com/news/press/blackstone-leads-funding-of-over-1-billion-to-neysa-to-work-towards-building-indias-leading-ai-infrastructure-platform/))\n\n- GPU capacity and current footprint: Neysa currently has about 1,200 GPUs live and plans to deploy more than 20,000 GPUs over time as demand accelerates. ([techcrunch.com](https://techcrunch.com/2026/02/15/blackstone-backs-neysa-in-up-to-1-2b-financing-as-india-pushes-to-build-domestic-ai-compute/))\n\n- Neysa platform and product details: Neysa operates an AI Acceleration Cloud platform (Velocis) that integrates GPU-based infrastructure with AI optimization and security tools to train, fine-tune, and deploy AI workloads. ([blackstone.com](https://www.blackstone.com/news/press/blackstone-leads-funding-of-over-1-billion-to-neysa-to-work-towards-building-indias-leading-ai-infrastructure-platform/))\n\n- Ownership and control: Blackstone will hold a majority stake in Neysa following the investment, with Neysa’s leadership continuing to drive growth via its CEO Sharad Sanghi. ([techcrunch.com](https://techcrunch.com/2026/02/15/blackstone-backs-neysa-in-up-to-1-2b-financing-as-india-pushes-to-build-domestic-ai-compute/))\n\n- Policy and market context: The investment aligns with India’s sovereign AI ambitions and IndiaAI Mission, including favorable policy signals such as a data-center tax holiday announced in the budget. ([blackstone.com](https://www.blackstone.com/news/press/blackstone-leads-funding-of-over-1-billion-to-neysa-to-work-towards-building-indias-leading-ai-infrastructure-platform/))\n\n- Blackstone’s broader infra strategy: The Neysa deal fits Blackstone’s global push into data center and AI infrastructure, building on its investments in data-center platforms and specialized AI infra players. ([techcrunch.com](https://techcrunch.com/2026/02/15/blackstone-backs-neysa-in-up-to-1-2b-financing-as-india-pushes-to-build-domestic-ai-compute/))\n\n- Reported valuation context: Indian press reports pegged Neysa’s enterprise valuation around $1.4 billion, with some outlets noting valuation not disclosed by Neysa; the dynamic is reflected across sources. ([economictimes.indiatimes.com](https://economictimes.indiatimes.com/tech/funding/blackstone-leads-600-million-raise-in-ai-cloud-neysa-at-1-4-billion-enterprise-value/articleshow/128390049.cms))\n\n- Notable customers (context of traction): Neysa’s client base includes Justpay, Swiggy, and Perfios, underscoring demand from Indian enterprises and public-sector entities for sovereign AI compute. ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/business/india-business/blackstone-leads-1-2-billion-raise-for-homegrown-ai-firm-neysa/articleshow/128403035.cms))',
  'model_provider': 'openai'},
 {'event_id': '595_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- What is new: Surging AI memory demand is intensifying a global memory/chip supply squeeze, with AI data-center needs re-prioritizing memory production and driving higher memory costs and allocation constraints. ([businesstimes.com.sg](https://www.businesstimes.com.sg/international/global/memory-chip-squeeze-wreaks-havoc-markets-more-come/?utm_source=openai))\n\n- Why it matters: The shift in memory allocation toward AI workloads is reframing AI compute costs and driving capacity-planning decisions for hyperscalers and cloud providers. ([ibselectronics.com](https://www.ibselectronics.com/resources/news/ai-buildout-is-tightening-memory-supply-and-pricing-pressure-is-spreading-beyond-phones-and-pcs/))\n\n- Demand shift specifics: AI data centers are increasingly diverting memory capacity toward high-bandwidth memory (HBM) and server DRAM, tightening supply for conventional DRAM/NAND used by consumer devices and other sectors. ([ibselectronics.com](https://www.ibselectronics.com/resources/news/ai-buildout-is-tightening-memory-supply-and-pricing-pressure-is-spreading-beyond-phones-and-pcs/))\n\n- Foreseeable price impact (1Q26): TrendForce forecasts conventional DRAM contract prices rising about 55–60% QoQ in 1Q26, NAND Flash 33–38% QoQ, with server DRAM prices rising more than 60% QoQ. ([trendforce.com](https://www.trendforce.com/presscenter/news/20260105-12860.html?utm_source=openai))\n\n- Additional market signals: IDC described an “unprecedented” memory shortage in late 2025 with effects persisting into 2027, due to AI-data-center demand outstripping supply. ([ibselectronics.com](https://www.ibselectronics.com/resources/news/ai-buildout-is-tightening-memory-supply-and-pricing-pressure-is-spreading-beyond-phones-and-pcs/))\n\n- Evidence of downstream price moves: OEMs and consumers are already seeing higher memory costs (e.g., LPDDR5x shortages contributing to desktop PC price hikes; frameworked price changes cited by Reuters in early 2026). ([ibselectronics.com](https://www.ibselectronics.com/resources/news/ai-buildout-is-tightening-memory-supply-and-pricing-pressure-is-spreading-beyond-phones-and-pcs/))\n\n- 1Q26 memory-price trajectory: DRAM prices are expected to surge sharply, with conventional DRAM contract prices seen rising 90–95% QoQ in 1Q26 per DRAMeXchange TrendForce commentary; NAND prices also slated for double-digit increases. ([dramexchange.com](https://www.dramexchange.com/WeeklyResearch/Post/2/12604.html?utm_source=openai))\n\n- HBM/AI-memory focus and capacity reallocation: TrendForce and industry analyses note growing AI/HBM demand is accelerating capacity shifts from consumer/mobile memory to AI-grade memory, with HBM supply prioritized over general DRAM/NAND. ([trendforce.com](https://www.trendforce.com/presscenter/news/20260105-12860.html?utm_source=openai))\n\n- Market consequences and investor/industry response: The memory squeeze has produced broad market disruption, with memory-sector equities rallying and major players reassessing pricing, allocation, and long-term procurement strategies. ([businesstimes.com.sg](https://www.businesstimes.com.sg/international/global/memory-chip-squeeze-wreaks-havoc-markets-more-come/?utm_source=openai))\n\n- Outlook through 2026–2027: Analysts expect the tight memory supply and price volatility to persist as AI infrastructure buildout continues to outpace new memory-capacity additions, sustaining elevated pricing power for memory suppliers. ([ibselectronics.com](https://www.ibselectronics.com/resources/news/ai-buildout-is-tightening-memory-supply-and-pricing-pressure-is-spreading-beyond-phones-and-pcs/))',
  'model_provider': 'openai'},
 {'event_id': '596_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Core development: UK announces at the India AI Impact Summit (February 16, 2026) that it will champion AI to supercharge growth, unlock new jobs, and improve public services globally. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))\n\n- Leadership and participation: The Deputy Prime Minister, David Lammy, and AI Minister, Kanishka Narayan, lead the UK delegation, with plans for engagement on how AI can improve everyday life and governance. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))\n\n- Funding context: The announcement is part of the UK’s AI for Development (AI4D) programme, described as £58 million in total funding, with new initiatives being announced at the summit. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))\n\n- Asian AI4D Observatory: One of the three new AI4D initiatives, designed to support responsible AI innovation and governance across South and Southeast Asia. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))\n\n- AI4D Compute Hub: Another new AI4D initiative, to establish a Compute Hub at the University of Cape Town to provide compute power for African innovators. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))\n\n- African Language Hub: UK will announce new support for an African Language Hub, enabling AI to operate in 40 African languages. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))\n\n- Summit context: The UK frames this as part of ongoing collaboration with India and broader governance efforts, noting ties to Vision 2035 and recent high-level engagement. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))\n\n- Global governance and prior summits: The press release situates the India summit within a sequence that includes the Bletchley Park AI Safety Summit and previous summits in Seoul and Paris, linking policy dialogue to governance and safety standards. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))\n\n- Private investment context: The UK notes that, since summer 2024, it has attracted more than £100 billion of private investment into the UK’s AI sector, underscoring the policy push to translate investment into jobs and growth. ([gov.uk](https://www.gov.uk/government/news/uk-to-champion-how-ai-can-supercharge-growth-unlock-new-jobs-and-improve-public-services-at-ai-impact-summit-in-india))',
  'model_provider': 'openai'},
 {'event_id': '611_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Core development: A February 16, 2026 Reuters market-reaction piece (widely republished) reports AI spending fears have driven a sell-off in major tech stocks, erasing hundreds of billions in market value for firms including Microsoft, Amazon, Nvidia, Apple, and Alphabet. ([channelnewsasia.com](https://www.channelnewsasia.com/business/big-tech-stocks-lose-billions-ai-spending-fears-hit-valuations-5934031))\n\n- Microsoft effect: Microsoft shares are down about 17% year-to-date, wiping roughly $613 billion off its market value to around $2.98 trillion as of February 13, 2026, amid concerns about AI profitability and rising competition from rivals like Google and Anthropic. ([channelnewsasia.com](https://www.channelnewsasia.com/business/big-tech-stocks-lose-billions-ai-spending-fears-hit-valuations-5934031))\n\n- Amazon effect: Amazon has shed about 13.85% year-to-date, erasing roughly $343 billion in market value and leaving it valued at about $2.13 trillion; the company also signaled capex growth of more than 50% this year. ([channelnewsasia.com](https://www.channelnewsasia.com/business/big-tech-stocks-lose-billions-ai-spending-fears-hit-valuations-5934031))\n\n- Nvidia, Apple, Alphabet: Since the start of 2026, Nvidia’s, Apple’s and Alphabet’s market values declined by roughly $89.67 billion, $256.44 billion, and $87.96 billion respectively, with current valuations around Nvidia $4.44 trillion, Apple $3.76 trillion, and Alphabet $3.70 trillion. ([channelnewsasia.com](https://www.channelnewsasia.com/business/big-tech-stocks-lose-billions-ai-spending-fears-hit-valuations-5934031))\n\n- Alphabet capex context: Alphabet indicated up to about $185 billion in capital expenditure for 2026, underscoring ballooning AI-related spending that underpins investor concerns about near-term returns. ([investing.com](https://www.investing.com/news/stock-market-news/sp-nasdaq-futures-subdued-as-markets-digest-alphabets-ai-spending-plans-4487019?utm_source=openai))\n\n- Market psychology shift: The pullback is described as a broader shift in investor sentiment from rewarding long-term AI initiatives to demanding clearer near-term earnings visibility. ([channelnewsasia.com](https://www.channelnewsasia.com/business/big-tech-stocks-lose-billions-ai-spending-fears-hit-valuations-5934031))\n\n- Other tech winners/losers frame: While Big Tech softened, some peers like TSMC, Samsung Electronics, and Walmart reportedly added market value during the period, illustrating a rotation within tech/tech-adjacent stocks. ([marketscreener.com](https://www.marketscreener.com/news/big-tech-stocks-lose-billions-as-ai-spending-fears-hit-valuations-ce7e5ddbd180f72d?utm_source=openai))\n\n- Publication framing/date: The Reuters-derived report was published February 16, 2026, with the cited figures reflecting the prior Friday, February 13, 2026, as the point of reference for “year-to-date” declines. ([channelnewsasia.com](https://www.channelnewsasia.com/business/big-tech-stocks-lose-billions-ai-spending-fears-hit-valuations-5934031))\n\n- Source note: All figures and claims above trace back to Reuters reporting on Feb 16, 2026, as summarized by CNA (Reuters) and corroborated by additional Reuters mirrors. ([channelnewsasia.com](https://www.channelnewsasia.com/business/big-tech-stocks-lose-billions-ai-spending-fears-hit-valuations-5934031))',
  'model_provider': 'openai'},
 {'event_id': '613_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Ant Group released Ling-2.5-1T, a trillion-parameter general-purpose LLM, and Ring-2.5-1T, the world’s first hybrid linear-architecture thinking model, on February 16, 2026; both are available under open licenses on Hugging Face and ModelScope. ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))\n- Ling-2.5-1T supports context lengths of up to 1,000,000 tokens and is designed for higher reasoning efficiency, fine-grained preference alignment, and native agent interaction. ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))\n- On AIME 2026, Ling-2.5-1T reportedly matches frontier thinking models that typically require 15,000–23,000 tokens while using roughly 5,890 tokens. ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))\n- Ring-2.5-1T is built on a hybrid linear architecture and targets advanced reasoning tasks; it reportedly achieved gold-tier results on IMO 2025 (35/42) and CMO 2025 (105/126). ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))\n- Ming-Flash-Omni-2.0, a large-scale omni model unifying speech, audio, and music within a single architecture, was released by Ant Group on February 11, 2026. ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))\n- Ling-2.5-1T and Ring-2.5-1T represent the latest evolution of Ant Group’s Ling 2.0 series, unveiled in October 2025. ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))\n- The Ling family includes Ling non-thinking MoE LLMs, Ring thinking models, and Ming multimodal models, marking a comprehensive upgrade across the Ling (BaiLing) model family. ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))\n- Both Ling-2.5-1T and Ring-2.5-1T are now openly licensed and hosted on Hugging Face and ModelScope, under the InclusionAI umbrella. ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))\n- Release location and date: Hangzhou, China, February 16, 2026, with the Business Wire-distributed press release detailing the open licensing and model capabilities. ([businesswire.com](https://www.businesswire.com/news/home/20260215551663/en/))',
  'model_provider': 'openai'},
 {'event_id': '615_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- GLM-5 is a 745B-parameter frontier LLM released as open-source (MIT license) by Zhipu AI, built as a Mixture-of-Experts model with 256 experts and 8 active per token, featuring 44B active parameters per inference and a 200K-token context window. It uses DeepSeek Sparse Attention and was trained on Huawei Ascend hardware with MindSpore. ([glm5.net](https://glm5.net/))\n\n- The model was publicly released in February 2026 (reported Feb 11–12, 2026 in China press and by international outlets), with open weights available on Hugging Face and GitHub and the release framed as a major open-source milestone. ([scmp.com](https://www.scmp.com/tech/article/3343239/chinas-zhipu-ai-launches-new-major-model-glm-5-challenge-its-rivals))\n\n- GLM-5’s weights are openly accessible under MIT license on Hugging Face (zai-org/GLM-5) and are downloadable for commercial use and local deployment; the model card also confirms the MIT license. ([huggingface.co](https://huggingface.co/zai-org/GLM-5))\n\n- The release specifies 28.5 trillion tokens of pre-training data, substantially larger than prior GLM iterations, and it notes a corresponding scale-up from 355B (GLM-4.7) to 744B parameters. ([globenewswire.com](https://www.globenewswire.com/de/news-release/2026/02/13/3238012/0/en/index.html))\n\n- GLM-5 introduces DeepSeek Sparse Attention (DSA) to improve long-context processing efficiency and reduce deployment costs relative to dense attention. ([scmp.com](https://www.scmp.com/tech/article/3343239/chinas-zhipu-ai-launches-new-major-model-glm-5-challenge-its-rivals))\n\n- A native “Agent Mode” is highlighted, enabling autonomous prompt transformation and source-material-to-output workflows (e.g., generating professional documents in .docx/.pdf/.xlsx formats). ([globenewswire.com](https://www.globenewswire.com/de/news-release/2026/02/13/3238012/0/en/index.html))\n\n- Benchmarking signals position GLM-5 as competitively strong among open models for coding and agentic tasks, with SWE-bench Verified scores around 77.8% and BrowseComp scores around 75.9, alongside strong results on other agentic benchmarks. ([huggingface.co](https://huggingface.co/zai-org/GLM-5))\n\n- In internal testing, GLM-5 reportedly surpasses Gemini 3 Pro on coding/agentic tasks and approaches Claude Opus 4.5 in certain metrics, though Anthropic’s Claude remains ahead on some coding benchmarks according to the release materials. ([scmp.com](https://www.scmp.com/tech/article/3343239/chinas-zhipu-ai-launches-new-major-model-glm-5-challenge-its-rivals))\n\n- Availability includes access through Z.ai’s platform and WaveSpeed API, with open weights published to Hugging Face, enabling community-led hosting and experimentation. ([glm5.net](https://glm5.net/))',
  'model_provider': 'openai'},
 {'event_id': '617_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Qureos raises $5 million in a seed round to accelerate its AI-driven hiring platform, with the round closed on February 4, 2026 in Dubai, UAE. Lead investors are Prosus Ventures and Salica Oryx Fund. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed))\n- The funds are intended to deepen Qureos’ AI capabilities and hiring workflows, expand the go-to-market team, and accelerate geographic expansion through enterprise and agency partnerships. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed))\n- The seed round includes participation from Oraseya Capital, PlusVC, F6 Ventures, BDev Ventures, Sunny Side Venture Partners, and Daniel Tyre, with follow-on investments from COTU Ventures and Globivest. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed))\n- Qureos’ product center is Iris, an AI recruiter assistant that automates sourcing, screening, and interviewing; the platform taps into 2,000+ job boards and can screen candidates in under 15 seconds. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed))\n- The founders are Alexander Epure (CEO) and Usama Nini (Co-founder). ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed))\n- Qureos reports traction with more than 1,000 enterprise and public-sector organizations; notable clients include Qatar Airways, Dubai Economy and Tourism, and Union Properties. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed))\n- The platform is UAE-founded and oriented to the Gulf region (MENA) but operates globally, with stated expansion to the United States. ([qureos.com](https://www.qureos.com/updates/qureos-raises-5-million-seed))\n- Independent coverage corroborates the seed round: CB Insights lists a $5M seed led by Prosus Ventures in February 2026. ([cbinsights.com](https://www.cbinsights.com/investor/prosus-ventures?utm_source=openai))',
  'model_provider': 'openai'},
 {'event_id': '618_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Farsight Vision announced a €7.2 million seed financing round to accelerate product development and scale delivery of its AI vision platform (announced Feb 13, 2026). ([kyivpost.com](https://www.kyivpost.com/post/70012?utm_source=openai))\n\n- Lead investors in the round were Axon Enterprise (USA) and Estonia’s SmartCap Defence Fund. ([kyivpost.com](https://www.kyivpost.com/post/70012?utm_source=openai))\n\n- Other participants included Radix Ventures (Poland), Anker Capital (Switzerland), Final Frontier (Denmark), with Resist.UA (Ukraine) and a follow-on from Darkstar (Estonia). ([kyivpost.com](https://www.kyivpost.com/post/70012?utm_source=openai))\n\n- The seed round amount (€7.2M) equates to roughly $8.6M at the time of reporting. ([kyivpost.com](https://www.kyivpost.com/post/70012?utm_source=openai))\n\n- Funds will be used to automate defense robotic systems for logistics, navigation, and weapons control; deepen integration with UAV/UGV manufacturers and battle-management systems; and enhance electronic warfare asset deployments. ([kyivpost.com](https://www.kyivpost.com/post/70012?utm_source=openai))\n\n- Farsight Vision operates as a Ukrainian-Estonian company with a footprint in Ukraine and Estonia; the Tallinn incorporation occurred in July 2024, and Brave1-cluster ties are noted. ([kyivpost.com](https://www.kyivpost.com/post/70012?utm_source=openai))\n\n- The round supports expansion beyond Ukraine, including partnerships across NATO countries and its first commercial contract in Asia. ([kyivpost.com](https://www.kyivpost.com/post/70012?utm_source=openai))\n\n- The platform provides real-time 3D terrain modeling from drone data and serves as an integration layer for mission planning, UAV/UGV coordination, and GNSS-denied environments. ([ukrinform.net](https://www.ukrinform.net/rubric-economy/4090812-ukrainianestonian-defense-startup-raises-eur-72m-in-investment.html?utm_source=openai))\n\n- The investment underscores ongoing European defense-tech funding activity, with involvement from public-safety and defense-focused investors and government-related support. ([kyivpost.com](https://www.kyivpost.com/post/70012?utm_source=openai))',
  'model_provider': 'openai'},
 {'event_id': '627_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Ricursive Intelligence disclosed a $300 million Series A at a $4 billion post-money valuation, bringing total funding to $335 million (including a $35 million seed) and confirming a public launch just months earlier. ([prnewswire.com](https://www.prnewswire.com/news-releases/ricursive-intelligence-raises-300-million-series-a-at-4-billion-valuation-to-accelerate-ai-driven-semiconductor-design-302670061.html))\n\n- The Series A was led by Lightspeed Venture Partners, with participation from DST Global, NVentures (NVIDIA’s VC arm), Felicis Ventures, 49 Palms Ventures, Radical AI, and Sequoia Capital. ([prnewswire.com](https://www.prnewswire.com/news-releases/ricursive-intelligence-raises-300-million-series-a-at-4-billion-valuation-to-accelerate-ai-driven-semiconductor-design-302670061.html))\n\n- The round occurred roughly two months after Ricursive’s public debut, which followed a seed round announced in December 2025. ([techcrunch.com](https://techcrunch.com/2026/01/26/ai-chip-startup-ricursive-hits-4b-valuation-two-months-after-launch/))\n\n- Seed funding: Ricursive Intelligence announced a $35 million seed round in December 2025 led by Sequoia Capital, valued at about $750 million at close. ([wvnews.com](https://www.wvnews.com/news/around_the_web/partners/pr_newswire/subject/new_products_services/ricursive-intelligence-launches-frontier-ai-lab-to-transform-semiconductor-design-and-accelerate-path-toward-artificial/article_dc7c4516-de58-5ad4-a8cd-16e25b9ce3d8.html))\n\n- Founders and background: Co-founders Anna Goldie (CEO) and Azalia Mirhoseini (CTO) previously worked together at Google Brain and were involved in AlphaChip, a project used in Google TPUs. ([techcrunch.com](https://techcrunch.com/2026/02/16/how-ricursive-intelligence-raised-335m-at-a-4b-valuation-in-4-months/))\n\n- Core product focus: Ricursive is building an AI-driven platform to design and iteratively improve AI chips, aiming to close the recursive loop between AI models and the hardware that runs them. ([prnewswire.com](https://www.prnewswire.com/news-releases/ricursive-intelligence-raises-300-million-series-a-at-4-billion-valuation-to-accelerate-ai-driven-semiconductor-design-302670061.html))\n\n- Technical scope: The platform is described as handling chip design tasks from component placement to design verification, and it envisions generating a silicon substrate through AI-driven workflows. ([techcrunch.com](https://techcrunch.com/2026/02/16/how-ricursive-intelligence-raised-335m-at-a-4b-valuation-in-4-months/))\n\n- Market position and customers: Nvidia is an investor, and the company’s stated target customers are chip makers and semiconductor-technology firms seeking automated, accelerated chip design. ([techcrunch.com](https://techcrunch.com/2026/01/26/ai-chip-startup-ricursive-hits-4b-valuation-two-months-after-launch/))\n\n- Location and status: Ricursive Intelligence is based in Palo Alto, California, described as a frontier AI lab advancing AI-driven semiconductor design. ([prnewswire.com](https://www.prnewswire.com/news-releases/ricursive-intelligence-raises-300-million-series-a-at-4-billion-valuation-to-accelerate-ai-driven-semiconductor-design-302670061.html))',
  'model_provider': 'openai'},
 {'event_id': '630_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Core development: Alibaba releases Qwen3.5-397B-A17B, the first open-weight flagship in the Qwen3.5 family, designed for agentic tasks with multimodal inputs; only about 17 billion of the total 397 billion parameters are active per forward pass. ([deeplearning.ai](https://www.deeplearning.ai/the-batch/qwen-announces-new-open-weights-flagship-update/?utm_source=openai))\n\n- Hybrid architecture: Qwen3.5 uses a hybrid design that combines sparse Mixture-of-Experts with linear attention via Gated Delta Networks to boost inference speed. ([dig.watch](https://dig.watch/updates/qwen3-5-397b-multimodal-ai-model-launch?utm_source=openai))\n\n- Multimodal inputs: The model is a native vision-language system supporting text, image, and video inputs, enabling multimodal reasoning and interaction. ([dig.watch](https://dig.watch/updates/qwen3-5-397b-multimodal-ai-model-launch?utm_source=openai))\n\n- Language expansion: Qwen3.5 expands multilingual coverage from 119 to 201 languages and dialects. ([dig.watch](https://dig.watch/updates/qwen3-5-397b-multimodal-ai-model-launch?utm_source=openai))\n\n- Context window: Qwen3.5-Plus offers a hosted option with a 1,000,000-token context window (native Qwen3.5 models expose substantial context capabilities as part of the platform). ([dig.watch](https://dig.watch/updates/qwen3-5-397b-multimodal-ai-model-launch?utm_source=openai))\n\n- Open weights availability: Weights for Qwen3.5 are openly available on HuggingFace and ModelScope for developers and organizations. ([deeplearning.ai](https://www.deeplearning.ai/the-batch/qwen-announces-new-open-weights-flagship-update/?utm_source=openai))\n\n- Efficiency and cost: Alibaba states Qwen3.5 is ~60% cheaper to run and delivers eightfold efficiency on large workloads relative to prior generations. ([mlq.ai](https://mlq.ai/news/alibaba-launches-qwen-35-ai-model-with-superior-efficiency-and-agentic-features/?utm_source=openai))\n\n- Hosted tooling and ecosystem: The Qwen3.5-Plus hosting via Alibaba Cloud Model Studio provides tool-use capabilities and large-context support for agentic workflows. ([dig.watch](https://dig.watch/updates/qwen3-5-397b-multimodal-ai-model-launch?utm_source=openai))\n\n- Additional technical context: Alibaba emphasizes asynchronous reinforcement learning and FP8 data pipelines as part of the training/inference efficiency story. ([dig.watch](https://dig.watch/updates/qwen3-5-397b-multimodal-ai-model-launch?utm_source=openai))',
  'model_provider': 'openai'},
 {'event_id': '631_2026-02-16',
  'news_date': '2026-02-16',
  'output': '- Medicomp Systems unveiled a Model Context Protocol (MCP) layer to safely expose its APIs to AI models, enabling context-aware, clinically grounded AI workflows (diagnostic prompting, chart summarization, coding crosswalks, etc.). ([medicomp.com](https://medicomp.com/medicomp-systems-unveils-new-ai-enablement-tools-focused-on-delivering-clinically-validated-evidence-based-intelligence/))\n\n- The MCP layer grounds AI outputs in Medicomp’s clinical knowledge foundation, leveraging the Quippe Clinical Knowledge Graph and the MEDCIN graph database to enable clinically validated reasoning and prevent unvalidated data from being saved or exchanged. ([medicomp.com](https://medicomp.com/medicomp-systems-unveils-new-ai-enablement-tools-focused-on-delivering-clinically-validated-evidence-based-intelligence/))\n\n- Core elements include clinically validating outputs from conversational and ambient AI, recalling and manipulating data in clinical notes via natural language, and converting narrative text into trustworthy structured data across medical domains. ([medicomp.com](https://medicomp.com/medicomp-systems-unveils-new-ai-enablement-tools-focused-on-delivering-clinically-validated-evidence-based-intelligence/))\n\n- The MCP architecture aims to reduce variability and risk of unstructured AI output by removing PHI access from large language models and embedding AI interactions within existing clinical workflows. ([medicomp.com](https://medicomp.com/medicomp-systems-unveils-new-ai-enablement-tools-focused-on-delivering-clinically-validated-evidence-based-intelligence/))\n\n- Live capabilities highlighted by Medicomp include recalling lab results, filtering entire charts by problem, and analyzing charts against quality measures using voice commands or natural language. ([medicomp.com](https://medicomp.com/medicomp-systems-unveils-new-ai-enablement-tools-focused-on-delivering-clinically-validated-evidence-based-intelligence/))\n\n- Medicomp ties these capabilities to its Alchemy product, described as the first tooling to validate, clean, and normalize clinical data at scale, now integrated with MCP-enabled workflows. ([medicomp.com](https://medicomp.com/medicomp-systems-unveils-new-ai-enablement-tools-focused-on-delivering-clinically-validated-evidence-based-intelligence/))\n\n- Select MCP-enabled interactions are already live with customers, with broader expansion planned for 2026 and beyond, and live previews scheduled for ViVE 2026 and HIMSS26. ([medicomp.com](https://medicomp.com/medicomp-systems-unveils-new-ai-enablement-tools-focused-on-delivering-clinically-validated-evidence-based-intelligence/))\n\n- Quippe’s clinical data foundation rests on MEDCIN’s clinical knowledge graph, which comprises over 430,000 clinical concepts and hundreds of millions of relevancy links, plus about 10 million code mappings, enabling AI-enabled workflows across care domains. ([medicomp.com](https://medicomp.com/quippe-clinical-data-engine?utm_source=openai))',
  'model_provider': 'openai'}]

In [41]:
def save_research(research_json):
    supabase.table('research_assistant').insert({
        'event_id': research_json['event_id'],
        'model_provider': research_json['model_provider'],
        'news_date': research_json['news_date'],
        'output': research_json['output']
    }).execute()

In [42]:
for i in research_output_6:
    save_research(i)
    print("✅ Saved")

✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
✅ Saved
